In [1]:
from electric_meter import *
from datetime import datetime
from IPython.display import HTML

In [17]:
meter = read_data('data/meter2.csv')

In [18]:
#meter = read_data('data/meter.csv')

costs, k, d = compute_costs(meter)

In [4]:
#costs.sort_index(ascending=True, inplace=True)

#from datetime import datetime
#costs.index = pd.to_datetime(costs.index).to_series().dt.strftime('%B %Y')

In [19]:
costs

,no_discount,pazgas_daytime,pazgas_unlimited,pazgas_weekend,pazgas_nighttime,amisragas_unlimited,electra_power,electra_hitec
timeperiod,,,,,,,,
יולי 2023,712.886993,676.627228,677.242643,686.250248,680.550604,666.549338,677.242643,671.618491
אוגוסט 2023,598.992556,571.751500,569.042928,576.796140,568.103026,560.058040,569.042928,564.450700


In [21]:
result = style_table(costs)
display(result)

,חחי,פזגז יום,פזגז ללא הגבלה,פזגז סופ״ש,פזגז לילה,אמישראגז,אלקטרה פאואר,אלקטרה הייטק
תקופה,,,,,,,,
יולי 2023,712.89,676.63,677.24,686.25,680.55,666.55,677.24,671.62
אוגוסט 2023,598.99,571.75,569.04,576.80,568.10,560.06,569.04,564.45


In [23]:
meter.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1404 entries, 2023-06-25 16:00:00 to 2023-08-23 03:00:00
Freq: H
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   consumption  1404 non-null   float64
 1   date         1404 non-null   object 
 2   wday         1404 non-null   int64  
 3   hour         1404 non-null   int64  
 4   wday_name    1404 non-null   object 
 5   timeperiod   1404 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 76.8+ KB


In [14]:
cols = [costs.columns[i] for i in [0,1,5, 6]]
display(style_table(costs[cols].copy()))

,חחי,פזגז יום,אמישראגז,אלקטרה פאואר
תקופה,,,,
יולי 2023,576.25,546.75,538.80,547.44
אוגוסט 2023,430.85,406.54,402.85,409.31
